## 1. Import Library and define function 

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)

In [2]:
def get_clf_eval(y_test, y_pred=None):
    confusion = pd.DataFrame(confusion_matrix(y_test, y_pred), index = ['T[0]', 'F[1]'], columns = ['pred_T[0]', 'pred_F[1]'])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [3]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx + 1
    series = series.map(my_dict)

    return series

### 1.1 load data

In [5]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv")
df_all = pd.concat([df_train, df_test], axis = 0)
df_all.shape, df_train.shape, df_test.shape

((64570, 30), (59299, 29), (5271, 30))

## 2. Data preprocessing

### 2.1 drop columns

In [6]:
# 열 삭제
drop_col = ['customer_country.1', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver',
            'product_subcategory', 'product_modelname', 'business_area', 'business_subarea', 'ver_cus', 'ver_pro']

df_all.drop(columns = drop_col, inplace = True)

### 2.2 결측치 처리(수치형 데이터)

In [7]:
# 결측값 0으로 넣을 컬럼
fillna_col = ['com_reg_ver_win_rate', 'historical_existing_cnt',
             'ver_win_rate_x', 'ver_win_ratio_per_bu']

for col in fillna_col:
    df_all[col] = df_all[col].fillna(0)

### 2.3 범주형 변수 전처리

customer_country : '/' 기준으로 나라만 추출 <br/>
customer_type : 비슷한 의미의 다른 용어를 같은 범주로 통일 <br/>
inquiry_type : 비슷한 의미의 다른 용어를 같은 범주로 통일 <br/>
customer_position : 비슷한 의미의 다른 용어를 같은 범주로 통일 <br/>
expected_timeline : 기간에 따라 5개의 범주로 나누고 기타와 None 범주 추가 <br/> <br/>
개수가 1개인 범주는 '기타' 범주로 통일 <br/>
결측치는 'None' 범주로 통일


In [8]:
df_all['customer_country'] = df_all['customer_country'].apply(lambda x: x.rsplit('/', 1)[-1].strip() if isinstance(x, str) else x)

In [9]:
df_all['customer_type'] = df_all['customer_type'].str.lower()
df_all['customer_type'] = df_all['customer_type'].replace(['etc.', 'other', 'others'], 'etc')
df_all['customer_type'] = df_all['customer_type'].replace(['end-customer', 'end customer', 'end-user'], 'end_user')
df_all['customer_type'] = df_all['customer_type'].replace(['specifier/ influencer', 'specifier / influencer'], 'influencer')
df_all['customer_type'] = df_all['customer_type'].replace(['homeowner', 'home owner'], 'home_owner')
df_all['customer_type'] = df_all['customer_type'].replace(['software/solution provider', 'software / solution provider'], 'solution_provider')
df_all['customer_type'] = df_all['customer_type'].replace(['engineer', 'hvac engineer'], 'engineer')
df_all['customer_type'] = df_all['customer_type'].replace(['distributor', 'dealer/distributor'], 'distributor')
df_all['customer_type'].fillna('none', inplace = True)

In [10]:
df_all['inquiry_type'] = df_all['inquiry_type'].str.lower()
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['etc.', 'other', 'other_', 'others'], 'etc')
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['sales inquiry', 'sales', 'probeam precio'], 'sales inquiry')
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['quotation or purchase consultation', 'request for quotation or purchase', 'quotation_or_purchase_consultation', 'purchase or quotation', 'quotation_', 'purchase'], 'quotation_or_purchase_consultation')
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['usage or technical consultation', 'technical consultation', 'request for technical consulting', 'usage_or_technical_consultation', 'technical_consultation', 'technical'], 'usage or technical consultation')
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['vui lòng báo giá giúp mình sản phẩm đo thân nhiệt xin cảm ơn', 'tôi cần tham khảo giá và giải pháp từ lg'], 'quotation_or_purchase_consultation')
df_all['inquiry_type'] = df_all['inquiry_type'].replace(['toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung'], 'product information')
df_all['inquiry_type'].fillna('none', inplace = True)

In [11]:
# custoper_position 전처리
df_all['customer_position'] = df_all['customer_position'].replace(['ceo/founder', 'partner', 'vice president', 'c-level executive', 'director', 'vicepresident', 'c-levelexecutive', 'vp','leadership/executive office/owner', 'president'
                                                                   'principal & director', 'business partner', 'chairman', 'co-founder', 'chief executive officer', 'subsidiary sales (ise)', 'ceo/fundador', 'gerente', 'the big boss',
                                                                  'principal & director', 'president'], 'ceo')
df_all['customer_position'] = df_all['customer_position'].replace(['consultant', 'commercial consultant', 'architecture/consult', 'architect/consultant'], 'consult')
df_all['customer_position'] = df_all['customer_position'].replace(['customer', 'customer_position'], 'customer')
df_all['customer_position'] = df_all['customer_position'].replace(['decision-influencer', 'decision maker', 'decision influencer'], 'decision')
df_all['customer_position'] = df_all['customer_position'].replace(['distributor', 'cargo'], 'distributor')
df_all['customer_position'] = df_all['customer_position'].replace(['physics teacher','assistant professor','maths lecturer','science teacher','guest faculty','physics faculty','teacher/middle school coordinator','prof.',
                                                                   'academic specialist','principal at oxford integrated pu science college','math and physics teacher','professor of mathematics','physics and mathematics teacher',
                                                                   'assistant professor of english','educator','professor','quantitative aptitude faculty','english trainer for ielts,toefl,pte,gre,sat exams.','associate professor',
                                                                   'pgt physics','education professional','chemistry teacher','director cum faculty at gaining apex coaching centre','teacher','senior lecturer',
                                                                   'neet/ olympiad expert faculty','associate professor in electronics engg','education','pgt chemistry', 
                                                                   'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)','assistant professor of enlish'], 'education')
df_all['customer_position'] = df_all['customer_position'].replace(['entry level','intern','trainee','entrylevel','employee'], 'employee')

df_all['customer_position'] = df_all['customer_position'].replace(['exhibition','exhibitiontv'], 'exhibition')
df_all['customer_position'] = df_all['customer_position'].replace(['hospital', 'medical device manufacturer', 'medical imaging specialist', 'tierarzt', 'surgery professional', 'pathologist'], 'medical')
df_all['customer_position'] = df_all['customer_position'].replace(['not applicable','none','other','others'], 'etc')
df_all['customer_position'] = df_all['customer_position'].replace(['manager','associate/analyst','consulting','lider de desarrollo','decision-maker','business unit director','business development','operations',
                                                                   'product management','market intelligence/research'], 'pm')
df_all['customer_position'] = df_all['customer_position'].replace(['asst prof.','professional trainer','radiology professional'], 'professional')
df_all['customer_position'] = df_all['customer_position'].replace(['this is a consume display requirement for home purpose.','unpaid','homeowner','no influence'], 'unemployed')

In [12]:
# value_counts 가 1개인 것들 etc로 분류
value_counts = df_all['customer_type'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_type'] = df_all['customer_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

value_counts = df_all['inquiry_type'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['inquiry_type'] = df_all['inquiry_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

value_counts = df_all['customer_position'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_position'] = df_all['customer_position'].apply(lambda x: 'etc' if x in values_to_replace else x)
df_all['customer_position'].fillna('none', inplace = True)

value_counts = df_all['customer_job'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_job'] = df_all['customer_job'].apply(lambda x: 'other' if x in values_to_replace else x)
df_all['customer_job'].fillna('none', inplace = True)

value_counts = df_all['product_category'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['product_category'] = df_all['product_category'].apply(lambda x: 'etc' if x in values_to_replace else x)
df_all['product_category'].fillna('none', inplace = True)

value_counts = df_all['customer_country'].value_counts()
values_to_replace = value_counts[value_counts == 1].index
df_all['customer_country'] = df_all['customer_country'].apply(lambda x: 'etc' if x in values_to_replace else x)
df_all['customer_country'].fillna('none', inplace = True)

In [13]:
expected_timeline = df_all['expected_timeline']
expected_timeline = expected_timeline.str.replace('-', '')
expected_timeline = expected_timeline.str.replace('_', '')
expected_timeline = expected_timeline.str.replace('~', '')
expected_timeline = expected_timeline.str.replace(' ', '')
expected_timeline = expected_timeline.str.replace('.', '')
expected_timeline = expected_timeline.str.replace('etc', 'others')

In [14]:
sort_df = expected_timeline.fillna('none')

In [15]:
values = ['aggressivepricerequired',
       'alreadycloseinjulypurchsewithrd', 'alreadyconnectwithpartner',
       'alreadyindiscussionwithpartnerfrombangalore(kohinoor)andwithmr.indraneelfromcorporateoffice',
       'alreadyintouchwithcustomer',
       'alreadyintouchwithcustomerfromlast10days.droppingitbecausewearealreadyintouch',
       'alreadyintouchwithhimsincelong.hehasnotfinalizedyet.leadforwardedtord',
       'alreadyintouchwithpartner.', 'alreadyintouchwiththereteam',
       'alreadyourcustomer,havesharedpricesfornewrequirement',
       'alreadypurchasewithrd', 'alreadysharedquotationthroughsi.',
       'alreadytouchwithcustomerforkioskrequirement',
       'alreadytouchwithcustomers',
       'alreadyworkingonidbwillcreateopppostclarfyingonsignage',
       'alreadyworkingwithenduseronthisrequirement.',
       'askedtocallbacklater,ihadsharedthisleadwithrdtotakeitfurther.',
       'askedtocallbacktomorrow,ihadsharedthisenquirywithrd.',
       'askedtocallbacktomorrowmorning,notresponding',
       'askedtocalllater.','(selectidtimeline)', '09022022requestedforboqofrequirement',
       '14:0015:00',
       '14thaugaggressivepricerequiredashehasgotpricefromoutsidedelhi',
       '14thfeb2022shareddetails.awaitingupdatefromcustomer.',
       '25nov2021nomobilenumber,sentamailtocustomer',
       '29thsep2021:nosuchrequirementasofnow', '14thfeb2022shareddetailsawaitingupdatefromcustomer','alreadyintouchwithcustomerfromlast10daysdroppingitbecausewearealreadyintouch',
       'alreadyintouchwithhimsincelonghehasnotfinalizedyetleadforwardedtord',
       'alreadyintouchwithpartner', 'alreadysharedquotationthroughsi',
       'alreadyworkingwithenduseronthisrequirement',
       'askedtocallbacklater,ihadsharedthisleadwithrdtotakeitfurther','alreadyindiscussionwithpartnerfrombangalore(kohinoor)andwithmrindraneelfromcorporateoffice',
       'askedtocalllater', 'askedtocallon4thmay',
       'askedtocontacthimafter8thjan', 'askedtosenddetails',
       'askedtosharedetailsonmailhewillupdateifthereisanyrequirement',
       'assignedtopartnerintialmeetingdonewillconverttoopppostcompleteinfo',
       'atpresentnotrequired,', 'beingfollowedup',
       'bodeli,requirementpendingnewquotesendcloseinthismonthend',
       'budgetissue', 'budgetlowhenceclienthavenotwentahead',
       'budgetproblem', 'busy,calllater', 'busyneedtocallback',
       'callafter3june', 'callanddiscusednorequirement',
       'callanddiscusedtocustomecustomerwantsdemo',
       'callbackagainon30/04', 'callbacklater',
       'callbacklater,infirstweekmarch', 'calledtoknowthepriceofidb',
       'callnotconnecting', 'callnotpicking',
       'clienthaveseenthedemoheneededforbiggerroomandthecameraqualityandspeakertrackingwerethemainrequiredfeatureasofnow,hisbudgetisaround2lacsandneedbiggersizesolutionthereisnoresponsefromclientpostdemohenceclosing',
       'clientiscrosscheckingonpricesitsarccaseofsubwayandthefranchiseeistryingtogetpricingfromelsewhere',
       'clientisexploringhewilldiscussoncedetailssentdetailsmailedtohimasperrecentdiscussion,heisnotexploringasofnow',
       'clientislookingfor86"displaywithvcsolution,theywillgofordemonextweekanddecideonprocuringourtr3dj,detailssharedwithclientonmail',
       'clientisnothavinganyrequirement,hewasonlybrowsingthroughtheproduthenceclosiginsystemalthoughthedetailsofidbaremailedtoclient',
       'clientisnotlookingnowandhewasinterestedwithonlyinbuiltopsoptionhenceclosinginsystemasheisnotrespondingtocalls',
       'clientisnotrespondingcalls,followingupandkeeptheupdatetracked',
       'clientnotanweringcallsandaskedthereisnorequirementhewastryingtocheckpricesforone65touchrequirementofexistingclient',
       'clientnotinterestedinproductreceingcallandnotansweringproperly',
       'clientshallgetbackforexploringdemoofidb,postthattheywilldecideonprocuringthesame',
       'clientwasbusy,askedtocallbacklater,interestedinexploringouridb',
       'clientwasdrivingandaskedtocallbacklater,willfolloupandkeepposted',
       'clientwasexploringonproductsbuthedonthaveanyplanstopurchasecurrentlyhehaveholdedonhisrequirementandwillconnectifitreiniitaties',
       'clientwasinquiringforpricesalreadyquotedbyfewpartnerstheyevenpurchasedasperrecentupdatebutclienthasnotconfiermedpartnername',
       'clientwaskeentoexploreouridbsku,infollowup', 'cmsreqired',
       'communicationnumbernotavailable',
       'concernedpersonnotavailable,willcalllater',
       'conference/meetingroom/collaborationspaces',
       'connectedwithpartnerforstudiosetupwithcamera',
       'connectingforissue', 'connectwithpartner',
       'contactdetailsprovidedarewrong,sentmailforcorrectcontactdetails',
       'convertedthisleadintoopportunity', "couldn'tconect",
       "couldn'tconnect", 'couldntconnect',
       'customehasnotreceivecallason31stmayspokewithcustomer,hewillcomefordemoinnextweek14062022',
       'customepurchaseconsumerproduct', 'customerasktocallbacktomorrow',
       'customerbudgetis125+taxfor75inchidbihavegivenournoregretofferandalsoexplainedwhylgisbetterandpremiumhesaidhewillcomebackinadayortwo',
       'customergettingpricefromjaipurwhichlowthanouroperatingprice',
       'customerhasbeennotansweringcall', 'customerhasnotansweringcall',
       'customerhasnotansweringcall/customerwant86"interactivedisplay,hewillpurchasewithinnext6month,weareforwardingtolocalrdtotakethisfurther',
       'customerhasnotansweringcall/quotationhasbeensendtocustomer',
       'customerisbusyaskustocallagain,following',
       'customerislookingforresidential43inchdisplayforhomeuse',
       'customerneedshortthrowprojector',
       'customernotansweringcall,willcallhimagainandupdate',
       'customerphoneisgetsswitchedoff',
       'customerpurchasebenqforpricedifferance',
       'customervisitatuvbusinessandseethedemowithin2to3days',
       'customerwantdemoofidbhewillcomefordemoinnextweekafterfollowingupwithcustomermultipletimesthereisnoupdateleadhasbeenforwardedtord',
       'customerwantproductt15000',
       'customerwillbecomingfordemoon28thmaycustomerdidnotcamefordemoihaveshareddetailswithrdtotakeitfurtherhencedroppingthislead',
       'customerwillcomefordemoinnextweek,thanhewilldecidefurther',
       'december2022', 'delhienquirey', 'demoalignedforclient',
       'democompleted,customeraskedtocomefeb10thforclosure', 'demodone',
       'demodoneanddetailsshared',
       'demoplanned,willupdatefurtherstatusonceitscompleted',
       'demoscheduledfor24thoct', 'demoscheduledforfirstweekfeb',
       'demotobealigned', 'demotobeplanned',
       'detailsharedwithhimonemailheislookingforoneunitof65inchforhisconferenceroom',
       'detailssend', 'detailssendquoterequire', 'detailssendrequiredemo',
       'detailsshared', 'detailsshared,beingfollowedup',
       'detailsshared,toplandemoindecember',
       'detailssharedfollowupunderprogress',
       'detailssharedfor75tr,followupinfirstweekofoct',
       'detailssharedfuunderprogress',
       "detailssharedonwhatsapp,he'llrevertback",
       'detailssharedwithconcernpersonwillaskbdotofollowup',
       'detailssharedwithcustomer,alsoihadsharedleadwithrdtotakeitfurther',
       'detailssharedwithcustomer,hewillupdateonfurtheractionafterevaluatingotherbrandsaswell',
       'detailssharedwithcustomerason4thmaytriedmultipletimesbutnotreachable',
       'detailssharedwithcustomeronmai',
       'detailssharedwithcustomeronmail',
       'detailssharedwithpartnerhehasquotedinourexistingcustomeronly',
       'detailssharedwithrdforfurtheraction,',
       'dicsussedwithclient,detailssharedonmail,clienthavenobudgetstobuynowhenceclosinginthesystem',
       "didn'tpickupthecall", "didn'trespond", "didn'trespondtocalls",
       "dind'trespond", 'discusedwithankitindehli',
       'discussedandshareddetailswithclientfordigitalsignagetheyarenotplanningtopurchsethesamecurrentlyhenceclosinginthesystem',
       'discussedwithclient,detailsmailedforidbandonequick,theyhavenoplantobuyfornowhenceclosinginthesystem',
       'discussedwithclient,detailsmailedtheyarenotplanningtopurchasefornowhenceclosinginthesystem',
       'discussedwithclient,detailssharedonmailclienthavenopurchaseplansfornowhenceclosingthesameinsystem',
       'discussedwithclient,heshallsendthedetailsoftheirofficerequirement',
       'discussedwithclient,quotesent',
       'discussedwithclient,theywerejustevaluatingtheproduct,theyhavenoplanstobuythesame,henceclosinginthesystem',
       'discussedwithclientdemoalignedoncedone,willupdate',
       'discussedwithclientdetailsmailedclientdonthavebudgetforpurchasenowhenceclosinginthesystem',
       'discussedwithclientdetailsmailedforonequick,theyarenotplanningtobuythesamefornowhenceclosinginthesystem',
       'discussedwithclientdetailsmailedtheyhavebudgeissuefornowsotheywontbuyhenceclosinginyhesystem',
       'discussedwithclientdetailsmailedtoclientnobudgetsfornowwontbuyhenceclosinginthesystem',
       'discussedwithclienttheyarelookingforaiosolutionwithvctheywillcallafter20thforthedemoandthenpurchasethroughgem',
       'discussedwithclienttheyarenotevaluatuatingidbasofnow,theyhavenoplanstobuyhenceclosinginthesystem',
       'discussedwithclienttheyhavenobudgetsforpurchasefornow,theywontbuyhenceclosinginthesystem',
       'discussedwithclienttheyneed24inchdisplayfordigitalsignagementionedforouravailablesizes,heshallrevert',
       'discussedwithclienttheyneedforcsractivitypricesexpectationisverymuchlow',
       'discussedwithclientweneedtoaligndemotheexpectationisconnectingappledevicesbutnotmandatoryoncedemoisdone,weshallupdate',
       'discussedwithclientwillplanfordemo',
       'discussedwithpartnerheworkswithcloudwalkerandwantedtoconnectwithusforsomeproductdetailsasofnow,donthaveanyinquirytoworkandhenceclosinginsystem',
       'discussedwiththeclient,tr3bg&onequickworksdemoscheduledon27thjuly2022postdemotheywilldecideonprocuringtheunit',
       'discussedwiththeclientaskedtocallbackonmondaywillkeepafollowup',
       'discussedwiththeclienthedonthavebudgetsdetailsmailedhewontbuynowhenceclosinginthesystem',
       'discussedwiththeclientpricessharedforonequickandidbonmailcustomertoconfirmaboutdemooftheskuandfinaliseonprocurement',
       'discussedwiththeclienttheyareintererstedinourtr3djseriesdemoplannedfor29thjuneposttheywillplanonprocurementwillkeepupdate',
       'discussedwiththeclienttheypurchased65um3dffromthechannelunabletotrackthesupplierhenceclosinginthesystemmailsenttoclientforourdisplayrange',
       'discussiontobedonewithcustomer',
       'donthaveanyimmediaterequirementhemaypurchaseafterjune',
       'donthavebudgethewantdotled', 'don’thavebudget',
       'drop,alreadydonewithpartner',
       'drop,budgetproblemnowphonenotrespondig',
       'drop,lookongforconsumerproduct', 'drop,notintrested',
       'drop,nownotintrested',
       'drop,requirementofinteractivetouchscreentosupplyingovt,conncgtwithpartner',
       'drop,stillnoplanforpurchase', 'drop,wrongnumber',
       'dropnoplanforpurchase',
       'dropnotwillingtobuynowwilldoinfuturedetailssharedonwhatsap',
       'dropped,requiresdevicelikeonequicksharetobeusedwithsamsungtv',
       'droppedcontactnonotvalid', 'droppedhavepurchasedcloudwalker',
       'duetobudgetissue,customerhasboughtmaxhubpanel',
       'duetofinancialreasons,clienthavedroppedpurchaseplanalthoughthedetailsofidbaremailedtoclient',
       'duplicatelead',
       'duplicateleadil220100042906discussedwithclienttheirvchallisunderdevelopmentonequickdetailsmailedtoclienttheywillcallusfordemoandpurchasefinalizationoncetheirvcplaceisready',
         'emailsendfordetailsclientisexploringonsolutionsbutonlyfortheirundertanding',
       'existingcaseworkedbyourdisti,crewbusiness,henceclosingasclientisonlycheckingforcommercials',
       'existingpartner,discussedandaddressedtheconcernthepartnerhadtakenpricingandwantedtorecheckasthemodelquotedischanged',
       'exitingcustomer',
       'financeandaccountingconsultantinquiredfoe43inchaiobutdonthaveplanstobuydetailsdiscussed',
       'followingup', 'forwardedtoaurangabadrdforfurtherprocess',
       'forwardedtobdo,beingfollowedup', 'forwardedtobdotofollowup',
       'fuunderprogress','heaskedtosharebudgetoryquote,hewillcheckandconfirmifhewillrequiretheproduct',
       'heclientisnothavinganyrequirementhenceclosiginsystemalthoughthedetailsofidbaremailedtoclient',
       'hehasaskedtoconnectnextweekhewanttoseethephysicaldemo',
       'heisaneventorganizationandlookingforrentaldisplayshencedroppinginsystem',
       'heiscomparingwithsenseswhichislocalbrand',
       'heisindiscussioninternallyandwillupdate',
       'heislookingforaninteractivedisplaydiscussedwithclient,willupdateasthecaseprogress',
       'heislookingforconsumerproductforhome','heislookingforinteractivedisplayquotesharedwithhim',
       'heislookingforvideowall&idbforhisofficeourbdojayantisintouchwithhimandsharingtherequireddetailstocustomer',
       'heisrequestingdemoinaligarhup,plzsharetoupperson',
       'hejustneedbudgetorypricingwehavesharedthedetailswithhimandrequestedhimtocomefordemo',
       'heneedsforhomepurposeandisaconsumerclientspokeandguidedonthedetailsandchannelnotab2brequirement',
       'hewant32inchtvforsignagedetailssharedwithhim',
       'hewant65inchat50k', 'hewant98inchinbelow2lacs',
       'hewantdemoinaurangabad,detailssharedwithrd',
       'hewantdemoinlaturandalsodonthavemuchbudgetpartnerfollowedupmultipletimesbutnoresponse',
       'hewantdemoinsangli,ihaveloopedwithlocalpartner',
       'hewantphysicaldemooftr3bfinnashikbuttherebudgetisalsoverylowsofirsthewilldiscusswithmanagementonbudgetthenconfirmbackfordemo',
       'hewillcallbackifneeded', 'hisbudgetisunder50konly',
       'hisbudgetisverylow',
       'hisbudgetisverylowbutwearetryingtoconvincehimfor65inch',
       'idbdetaissend',
       'il220100042906repeatinquiryhenceclosingthisinsystem',
       'invalidlead',
       'isaninteriordesignerworkingonturnkeyprojectsmrbhuvneshisincontactandsharedspecsandprices',
       'itsbiharcustomer',
       'kinnarieletronisvatwahasbidforthesameandtheyaregoingtopurchasefromthem',
       'knownpartnerandalreadyworkingonacaseonlycrosscheckingpriceshencedropping',
       'knownpartnerandwehaveinstalled3videowallspanindiaforcctvwesupplied49vl5batrs62000andthenagain49vl5fatrs71000nowtheavailablemodelis49vl5gat80000+whichpartnerisrecheckingandhencemadeinquiryclosingtheinquirysi',
       'leadbelongstomaharashtra','lookingforactiveledinarangeof5lakhsnotaprospetivebuyer',
       'lookingforairconditioners',
       'lookingforcommercialtv55"detailsmailedtoclient,theyhavenoplansfornowtobuyhenceclosinginthesystem',
       'lookingforhomeuse',
       'lookingfornotmadeinchinaactiveoutdoorledforgovernmentproject',
       'lookingforoutdoordisplaycusotmerhasnoclarityonthesolutionemailsharedforunderstandingandwaitingforreply',
       'lowbudget', 'maileddetails&priceof55inches',
       'mayurelectronicconnectwithclient', 'meetingdatetobefinalised',
       'meetingplannedforfurtherdiscussion','mobileisswitchedofftried3times','mrrajnikantisallignwiththeclient',
       'needforhomesegmenttheypurchasedvu85inchtvfortheirhome',
       'needtoarrangequoteanddemo', 'needtododomestictradebiz',
       'nextweekhewillcomefordemoatlgoffice',
       'nocontactnumber,ihavedroppedaemailrequestingforthesame',
       'nodropthislead,nobudget!',
       'nofurtherrevert&clarityaftersharingdetails', 'noreqirements',
       'norequirement', 'norequirementofthistime', 'noresponse',
       'noresponseonmailsfromtheclientcouldntfindanycontactdetailsonnetaswellhencedroppinginthesystemalthoughidbandonequickdetailssharedonemail',
       'notansweringcall', 'notansweringcall,detailssharedwithrd',
       'notansweringcall,leadsharedwithrd',
       'notansweringcall,salesremarks:triedtoreachhimmultipletimesbutheisnotrespondingrequesttoshailjatoreconnectwithcustomerwearedroppingthisleadfornow',
       'notansweringcall,sharedleadwithrd',
       'notansweringcall,willtrytoreachhimagain',
       'notansweringcall|ason11thjunecustomerphoneisnotreachableiamdroppingthisleadandforwardingtopartner',
       'notansweringtriedseveraltimesdetailssharedwithrd',
       'notinterested', 'notinterestedatthemoment',
       'notinterestedincommercialtvs', 'notlifted', 'notlifted,calllater',
       'notpickingcalls,calledmanytimes', 'notreachable',
       'notreachable,detailssharedwithrd',
       'notreachable,ihadsharedleadwithpartner',
       'notreachable,leadsharedwithpartner', 'notrequire', 'notrequired',
       'notrequirednow', 'notresponding',
       'notresponding,detailssharedwithlocalrd', 'notrespondingtocalls',
       'notrespondingtriedmanytimes', 'november2022', 'november22',
       'october2022','ordertaken',
       'ourpartner,visnetworksisworkingonthisinquiryandclientiscrosscheckingthepriceshenceclosinginthesystem',
       'partnerisalreadyintouchwithourrd,ornothepartnerneedstocrosscheckontheinformationasthesizeischangedfrom49to50nowclosingtheopprtunityasitsexistingwithrd',
       'partnerisintostaticsignagesheneedtoconnectfordigitalsignagebuthehimselfismanufactureralthoughamailinitaitedforanyrequirementswithledteamtotakeitaheadasofnow,thereisnoscopehenceclosinginsystem',
       "partnerisworkingonexistingrequirementheisconnectedwithourdisti'sforbilling",
       'partnerwasexploringoneolmodelforsomerequiremeentnodesignatedsalesorclosureexpectedhenceclosinginthesystem',
       'phonenumbernotavailable', 'phoneswtichedoff', 'priceshared',
       'pricesharedwithcustomerheislookingforsomeidbsneedtotakefollowup',
       'pricesharewithcustomerourrdisfollowingupwithhimbutcustomerisnotresponding',
       'projectcancle',
       'proposalsubmitted,duetocovidmdisnotcomintoofficeforthemeeting',
       'providedmodelandpriceasperrequirement',
       'purchasedfrombinary@123l', 'purchasedsomelocalpanel,budgetissue',
       'purchasedviewsonicpanel','purchasevubrand',
       'quotationsent–75tr3dj,workinprogress,hewillbuyafter2monthshehasnotevenseenthequoteyet',
       'quotationshared',
       'quotationsharedforultrastrothershand49vl5ghewillcheckwithmanagementandupdateus',
       'quotationsharedincommunication', 'quotationsharedwithcustomer',
       'quotationsharedwithcustomer,hewillconfirmbackafterdiscussingwithmanagement09062022asontoday14062022notansweringcallhenceiamdroppingthisleaddetailsshredwithrdtakeitforfurther',
       'quotationsharedwithhimforidbhedonthaveenoughfundasofnowheisjustevaluatingoptions',
       'quotehasbeensenttocustomer', 'quotesend',
       'quotesendmeetingpending', 'quotesendnextmonthrequire',
       'quotesendrequiredemo', 'quotesendrequiredemoinnextweek',
       'quotesent,theclientisrequireddemoinjune', 'quotesenttocustomer',
       'quotesharedbuttherebudgetisverylow',
       'quotesharedfor86inchheisevaluationlocalbrandoftrueview',
       'quotesharedwithcustomer,hewillconfirmafter2monthsleadsharedwithpartner',
       'quotesharedwithcustomer,leadsharedwithpartnertotakeitfurther',
       'quotesharedwithcustomerhewillrevertwithanupdate',
       'quotesharedwithhim',
       'quotesharedwithhimandhewillupdateusafterdemo',
       'rajnikantisworkinginthiscasehoinahmedabadanditsunderprocess',
       'receivedpo', 'recentlynorequirement,dropped',
       'remarks:customerlookingfor65inchesand75incheseach1qty',
       'repeatedinquiryfromclienthewasjustinquiringbuthavenotconfimredonpurchasewehavetriedmulitpletimesbutclientisnotinterestedinbuying',
       'repeatenquiry',
       'repeatinquiry,wealreadyknewontheclientrequirementweareintouchwiththemandifanyrequirementcomes,wewilludpateasofnow,thetransparentledtheyneedisnotavailablewithusandtheavailableproductcostexpectationisverylowwit',
       'require65inchidbunder80k', 'requiredemo', 'requiredemoafter20may',
       'requiredemopricesend',
       'requirementisforpuneloopedpuneteamalthoughthepartnerischeckingpricesforanexistingcaseclosingtheleadinsystemaspuneteamisalreadyworking',
       'requirementof65tr3dj',
       'requirementof65tr3djandcustomerwantdemo,detailssharedwithrd',
       'requirementof65tr3dj–demoreqired,leadsharedwithrd',
       'requirementof65tr3dj–demorequired',
       'requirementof75tr3dj–demo,detailssharedwithrd',
       'requirementof75tr3dj–demo,leadsharedwithmumbaiteam',
       'requirementof75tr3dj–demorequired',
       'requirementofidb,budgetisverylowshareddetailsonwhatsapp1unitrequiredrdisfollowingup',
       'requirementofvideowallclientischekingonpricesalready2partnersworkingitsongoingcasehenceclisinginsystem',
       'requireonlycmssoftware', 'requirequote',
       'requiresdetailfortendernopurchaserequirementrightnow',
       'resellerwanttohavegemauthorization',
       'reuiredforofficeaskedtosharequotation', 'rnr',
       'sameasleadnoil220300046498hencedroppingduplicatelead',
       'schedulingameeting', 'september', 'september2022',
       'sizenotavailable', 'smallsizetabrequired',
       'spokentoclient,hewillcheckiftheyneeddemoandconfirm',
       'spokentoclient,hewillconnectoncehediscussonrequirement',
       'spokentopartner,havenoanyrequierementsfornow,havemailedthedetailstohim,hewontpurchaseanyskusfornowhenceclosinginthesystem',
       'spokewithcustomehewant43"tv',
       'spokewithcustomer,detailsneedstoshareonmailalsotheywantdemo',
       'spokewithcustomer,heislookingfordotledproductin42inch,ihavesuggestedhimour43inchsignagebuthewantfullyoutdoorledihavesharedthedetailsofour43inchdisplay',
       'spokewithcustomer,willbesharingdetailsonemail',
       'spokewithherandsharedthedetailsforsignagedisplays','theclientbudgetiswithin1lacstheyneed55inchaiosolutionasperdiscussion,theyarecomparingwithjabraandweproposedtocheckiftheyneedonlydisplaysbutduetobudgothersontraint,theyaredroppingtheidea',
       'theclientislookingforsmallsizedisplaysof20inchto25inchtomountonrefthebudgetis4kto6knotexactlyourrequirement',
       'theclientisnothavinganyrequirement,hewasonlybrowsingthroughtheproduthenceclosiginsystemalthoughthedetailsofidbaremailedtoclient',
       'theclientisnothavinganyrequirement,shewasonlybrowsingthroughtheproduthenceclosiginsystemalthoughthedetailsofidbaremailedtoclient',
       'theclientisnothavinganyrequirementhenceclosiginsystemalthoughthedetailsofidbaremailedtoclient',
       'theclientisnothavinganyrequirementhewasjustbrowsingforproductshenceclosiginsystemalthoughthedetailsofidbandvideowallsaremailedtoclient',
       'thepartnerischeckingpriceforanongoingcaseofbarcwehavediscussedonnewcasewiththepartneritsusefultotakeourdiscussionfurtherfornewinquriesclosingtheopportunityinsystemasofnowasitsmadeonlyforcrosscheckingprices',
       'thererquirementisforpuneemailedclientloopingpuneteamtomeetandvisittheclient',
       'theyareconsultantandiamalreadyintouchwithcustomer',
       'theyarehavingrequirementof40displayscustomerneedaggressivepricingthansamsung',
       'treidmanytimesbutphonenotreachable',
       'triedcallingtheclient,noresponsefromclient,ihavemailedidbdetails,willkeepfollowup&update',
       "triedcontacthimbutcouldn'tabletoconnectleadforwardedtord",
       'triedcoupleoftimesbutheisnotrespondingwewilltryagain',
       'triedreachingouttoclientnoresponsetocalllssoihavesharedthedatasheet,shalllfolllllowupandupdate',
       'triedtoreachcustomerbutnoresponseleadforwardedtopartnerforfurtherfollowup',
       'triedtoreachhimbutnoresponseiwillcallhimagain18thnovicalledhimagainandhewillsendtherequirementonmail',
       "triedtoreachhimmultipetimesfromlast7daysbutcouldn'tgetthrough",
       "triedtoreachhimmultipletimes,butcouldn'tgetthru",
       'triedtoreachseveraltimesbutnoresponse', 'underdiscussion','uttarpradeshstatelead',
       'veryabruptcustomersaidtheinquirywasmademonthsagoandwasrudeenoughclosinginthesystemastheclientbehaviourhasnoscopetodiscussonrequirementneedmarketingteamtocheckifthecasewasreceivedindecorjanasperclient',
       'wanttodotradeproductbusiness', 'wanttotakeretailmonitor',
       'wearealreadyintouchwithcustomerthrupartner',
       'wearealreadyintouchwiththiscutsomerfromlast2years,hehasneverpurchasedanyproducttilldateicalledhimuptwicebutnoreponse',
       'wearenotgoingtomatchtechnicalspecification',
       'wehavepitched110inchfuunderprgress',
       'wewerealreadyworkinghere,wearefacingchallengeaswedonthave49vl5ginstockorderlostinmarchduetounavailabiltyof49vl5g',
       'willcomebackonceitisrequired', 'willcomeforthedemo',
       'withdrawnthedecissiontopurchase', 'últimarevisión:26/07/21','underdiscussion,requirementnotdinalisedyet',
       'update14thseptmoredetailstobesharedbyclient',
       'updateaspercustomernoenquirywasmade',
       'updatedetailsalreadysharedwithcustomerevaluationisongoing','uptodecember']

sort_df = sort_df.apply(lambda x: 'others' if x in values else x)
sort_df = sort_df.replace('callbacktomorrow28/09', 'lessthan3months')
sort_df = sort_df.replace('then', 'than')
len(sort_df.unique())

42

In [16]:
values = ['lessthan3months,meetingwiththecustomerforthemoredetailsandtentativeboqwillne32and43',
       'lessthan3monthscustomernotansweredtocallback',
       'lessthan3monthsoutdoorledrequiment', '4/8months', '45days', 'askedtocallbacktomorrow',
          'askedtocallnextweek','askedtocallbacktomorrow,ihadsharedthisenquirywithrd',
         'duplicateleadil220100042906lessthan3months', 'eolmodelnewmodelquoterequirmentafter30days',
         'getbackin1wk','heislookingfordisplayforhisrestauranthehasaskedtocontacthimafter1monthasrestaurantworkisunderprogress',
        'hewantdemonextmonthatpune15may', 'hewantdemonextweek','needtodiscusswithclientinnexttwomonthstheyneedtochecktheproductandaccridnglyproceedforpersonaluse'
          ,'meetingplannednextweek','needtocallonfriday27th', 'onemonth', 'spokewithshubhamwhoisstudenthesaidhewillspeakwithhissirandrevertbackafter2weeks', '3months',
         'update14thseptcustomerwilldiscussindetailtomorrow','update13thspetfollowuptobedoneon15thsept', 'update27thaugcustomerhastocomeforproductdemo',
       'update4thaugcustomeraskedtocallonmonday',
       'update4thaugcustomervisitingexperiencecentreon8thsept',
       'update4thaugdetaileddiscussiontobedoneon6thaug',
       'update7thaugdemogivencustomerwillconfirmnextweek',
       'update8thseptdetailssharedwillconfirmdemoscheduleofnextweek',
       'update9thoctdetailssharedwillconfirmforonlinedemonextweek', 'updatecalltobemadeontuesday',]

sort_df = sort_df.apply(lambda x: 'lessthan3months' if x in values else x)

In [17]:
sort_df = sort_df.replace('lessthen6months', 'lessthan6months')
sort_df = sort_df.replace('lessthan5months', 'lessthan6months')
sort_df = sort_df.replace('morethen3months', 'lessthan6months')
sort_df = sort_df.replace('needshoteltvafter4months,willcallus', 'lessthan6months')
sort_df = sort_df.replace('purchaseplanningafter3months', 'lessthan6months')
sort_df = sort_df.replace('lessthan6months', '3months6months')

In [18]:
df_all['expected_timeline'] = sort_df

In [19]:
df_all['expected_timeline'].unique()

array(['lessthan3months', 'none', '3months6months', '9months1year',
       'morethanayear', '6months9months', 'others'], dtype=object)

레이블 인코딩

In [20]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
    
df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):]

In [21]:
sample = df_train[df_train['customer_idx'] == 25096].sample(100, random_state=42)
df_train = pd.concat([df_train[df_train['customer_idx'] != 25096], sample])

In [22]:
df_true = df_train[df_train['is_converted'] == True] # 2529
df_false = df_train[df_train['is_converted'] == False] # 54449

In [23]:
# 데이터를 고르게 추출하기 위한 셔플
df_false = shuffle(df_false, random_state = 42)

In [25]:
ratio = df_false.shape[0] // 20
xgb_params = []
cat_params = []
lgbm_params = []
# total_xgb_test = pd.DataFrame()
# total_cat_test = pd.DataFrame()
# total_lgbm_test = pd.DataFrame()
total_voting_test = pd.DataFrame()

x_test = df_test.drop(["is_converted", "id"], axis=1)

for i in tqdm(range(20)):
    if i != 19:
        df_f = df_false.iloc[ratio*i:ratio*(i+1)]
    elif i == 19:
        df_f = df_false.iloc[ratio*i:]
    
    df_train = pd.concat([df_true, df_f])
    
    x_train, x_val, y_train, y_val = train_test_split(
        df_train.drop(["is_converted", 'id'], axis=1),
        df_train["is_converted"].astype(int),
        test_size=0.2,
        random_state=42
        )
    
    ## xgb
    xgb = XGBClassifier(random_state=42)
    param_grid = {'n_estimators' : np.arange(200, 1201, 100), 'learning_rate' : np.arange(0.01, 0.15, 0.02),
                  'subsample' : [0.8], 'colsample_bytree' : [0.8]}

    grid_xgb = GridSearchCV(xgb, param_grid, verbose = True, scoring='f1')
    grid_xgb.fit(x_train, y_train)
    
    xgb_params.append(grid_xgb.best_params_)
    
    test_pred = grid_xgb.predict_proba(x_test)
    # total_xgb_test = pd.concat([total_xgb_test, pd.DataFrame(test_pred)], axis = 1)

    ## lgbm
    lgbm = LGBMClassifier(random_state=42, verbose = 0)
    param_grid = {'n_estimators' : np.arange(200, 601, 100), 'learning_rate' : np.arange(0.03, 0.12, 0.02),
                  'subsample' : [0.8], 'colsample_bytree' : [0.8]}

    grid_lgbm = GridSearchCV(lgbm, param_grid, verbose = True, scoring='f1')
    grid_lgbm.fit(x_train, y_train)
    
    lgbm_params.append(grid_lgbm.best_params_)
    
    test_pred = grid_lgbm.predict_proba(x_test)
    # total_lgbm_test = pd.concat([total_lgbm_test, pd.DataFrame(test_pred)], axis = 1)

    ## catboost
    cat = CatBoostClassifier(random_state=42, verbose = False)
    param_grid = {'iterations' : np.arange(400, 1501, 100), 'learning_rate' : np.arange(0.05, 0.15, 0.02),
                  'subsample' : [0.8], 'colsample_bylevel' : [0.8]}

    grid_cat = GridSearchCV(cat, param_grid, verbose = True, scoring='f1')
    grid_cat.fit(x_train, y_train)
    
    cat_params.append(grid_cat.best_params_)
    
    test_pred = grid_cat.predict_proba(x_test)
    # total_cat_test = pd.concat([total_cat_test, pd.DataFrame(test_pred)], axis = 1)

    ## Voting
    vote = VotingClassifier(estimators=[('xgb', grid_xgb.best_estimator_),
                                        ('lgbm', grid_lgbm.best_estimator_),
                                        ('cat', grid_cat.best_estimator_)], voting='soft')

    vote.fit(x_train, y_train)
    test_pred = vote.predict_proba(x_test)
    total_voting_test = pd.concat([total_voting_tset, pd.DataFrame(test_pred)], axis = 1)

  0%|          | 0/20 [00:00<?, ?it/s]

Fitting 5 folds for each of 77 candidates, totalling 385 fits


  5%|▌         | 1/20 [04:33<1:26:44, 273.92s/it]

Fitting 5 folds for each of 77 candidates, totalling 385 fits


 10%|█         | 2/20 [09:08<1:22:14, 274.16s/it]

...

Fitting 5 folds for each of 77 candidates, totalling 385 fits


 45%|████▌     | 9/20 [41:07<50:06, 273.33s/it]

Fitting 5 folds for each of 77 candidates, totalling 385 fits


 45%|████▌     | 9/20 [42:52<52:24, 285.86s/it]


KeyboardInterrupt: 

In [ ]:
ff = total_voting_test.iloc[:, ::2].mean(axis = 1)
tt = total_voting_test.iloc[:, 1::2].mean(axis = 1)

result = []
for i in range(len(ff)):
    if ff[i] > tt[i]: result.append(0)
    elif ff[i] < tt[i]: result.append(1)
        
result = np.array(result)

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = result

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)